In [20]:
import os
import gc
import torch
import chromadb
from chromadb.utils import embedding_functions
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Pipeline)

# Cleanup

gc.collect()
torch.cuda.empty_cache()
# Configuration
DB_PATH = os.path.join(os.getcwd(), "arxiv")  # Safe path joining
MODEL_ID = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [21]:
# Initialize Client & Embedding
client = chromadb.PersistentClient(path=DB_PATH)

embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
collection = client.get_or_create_collection(name="Arxiv-Database", embedding_function=embedding_func
                                             )


def retrieve_context(query_text, n_results=3):
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )
    # Flatten the list of documents
    docs = results["documents"][0] if results["documents"] else []
    return docs

In [22]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)


TypeError: Qwen2ForCausalLM.__init__() got an unexpected keyword argument 'device'